In [ ]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')


In [ ]:
login = pd.read_csv('key.csv')
consumer_key = login['key'][0]
consumer_secret = login['secret'][0]
access_token = login['token'][0]
access_token_secret = login['token_secret'][0]

In [ ]:
#auth obj
authenticate = tweepy.OAuthHandler(consumer_key, consumer_secret)

#set access token
authenticate.set_access_token(access_token, access_token_secret)

#create api obj
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [ ]:
#Extract 100 tweets from twitter user
def getTweetFromUser(query,n,lang):
    return api.user_timeline(screen_name=query, count=n, lang=lang, tweet_mode="extended")

def getTweet(query,n):
    return api.search(q=query, count=n, tweet_mode="extended")

posts = getTweet("#ม็อบ18พฤศจิกา",50)


for tweet in posts[0:5]:
    print(tweet.full_text+'\n')

In [ ]:
#create dataframe
df = pd.DataFrame([tweet.full_text for tweet in posts], columns= ['Tweets'])

df.head()

In [ ]:
#clean the text

#create function to clean the tweets
def cleanText(text):
    text = re.sub(r'@[A-Za-z0-9_]+:', '', text) # remove #mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # remove #mentions
    text = re.sub(r'#','',text) # remove hashtag
    text = re.sub(r'RT[\s]+','', text) # remove RT
    text = re.sub(r'https?:\/\/\S+','', text) # remove hyper link
    return text

#clean the text
df['Tweets'] = df['Tweets'].apply(cleanText)

df

In [ ]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#create net col
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

In [ ]:
df

In [ ]:
df.to_csv(r'sentiment.csv', index = False)